# Fetch local data

In [1]:
import pandas as pd

In [12]:
dataset = pd.read_csv('car_indexed.csv')

In [23]:
len(dataset)

1728

In [14]:
dataset.iloc[0].to_dict()

{'index': 0,
 'buying': 'vhigh',
 'maint': 'vhigh',
 'doors': 'two',
 'persons': 'two',
 'lug_boot': 'small',
 'safety': 'low',
 'class': 'unacc'}

# DynamoDB table creation

In [11]:
import boto3
dynamodb = boto3.resource('dynamodb')

In [18]:
table = dynamodb.create_table(
    TableName='car_table',
    KeySchema=
    [
        {
            'AttributeName': 'index',
            'KeyType': 'HASH'
        }
    ],
    AttributeDefinitions=
    [
        {
            'AttributeName': 'index',
            'AttributeType': 'N'
        }
    ],
    ProvisionedThroughput=
    {
        'ReadCapacityUnits': 1,
        'WriteCapacityUnits': 1
    }
)

In [20]:
table.wait_until_exists()

In [25]:
with table.batch_writer() as batch:
    for i in range(len(dataset)):
        batch.put_item(
            Item=dataset.iloc[i].to_dict()
        )

# Fetch test

In [32]:
response = table.get_item(
    Key={
        'index': 50
    }
)

In [33]:
item = response['Item']
print(item)

{'index': Decimal('50'), 'buying': 'vhigh', 'maint': 'vhigh', 'safety': 'high', 'lug_boot': 'med', 'persons': 'more', 'class': 'unacc', 'doors': 'three'}
